In [66]:
import json
import pandas as pd

train_list = []
file_name = "test_time.txt"
with open(file_name, "r", encoding="utf-8") as f:
    for line in f:
        if line[0] in ["G", "D", "C", "S", "Z", "Y", "T", "K", "N"]:
            train_list.append({"train_id": "", "train_time": []})
            train_list[-1]["train_id"] = line.strip()
        elif line == "" or line == "\n":
            continue
        else:
            train_list[-1]["train_time"].append(line.strip().split("\t"))

with open("train_time.json", "w", encoding="utf-8") as f:
    json.dump(train_list, f, ensure_ascii=False, indent=4)

In [67]:
s = set()
for train in train_list:
    for stop in train["train_time"]:
        s.add(stop[0])
total = len(s)

df = pd.DataFrame(s).sort_values(by=0).reset_index(drop=True)
df.columns = ["station"]
df["station_id"] = range(0, total)
d={}
for i in range(total):
    d[df["station"][i]] = str(df["station_id"][i])
with open("station_id.json", "w", encoding="utf-8") as f:
    json.dump(d, f, ensure_ascii=False, indent=4)

In [68]:
station2id = {}
for id in range(0, total):
    station2id[df.iloc[id, 0]] = id

for train in train_list:
    for stop in train["train_time"]:
        stop.append(station2id[stop[0]])

with open("train_time.json", "w", encoding="utf-8") as f:
    json.dump(train_list, f, ensure_ascii=False, indent=4)

In [69]:
import datetime, math

INTERVAL = 15
SPLIT = 24 * 60 // INTERVAL
STATION_END = total * SPLIT * 2


def station_in(station_id):
    return station_id * SPLIT * 2


def station_out(station_id):
    return station_id * SPLIT * 2 + SPLIT


def transfer(station_id, time, io):
    time = datetime.datetime.strptime(time, "%H:%M")
    if io == 0:
        time = math.ceil((time.hour * 60 + time.minute) / 15) % SPLIT
    else:
        time = math.floor((time.hour * 60 + time.minute) / 15) % SPLIT
    return int(station_id * SPLIT * 2 + time + io * SPLIT)


file = open("in.txt", "w")
file.write(str(len(d.keys())) + "\n")
for i,v in d.items():
    file.write(str(i)+" "+str(v)+"\n")
# file.write(str(STATION_END + total)+"\n")
for train in train_list:
    for f in range(0, len(train["train_time"]) - 1):
        for t in range(f + 1, len(train["train_time"])):
            starttime = datetime.datetime.strptime(train["train_time"][f][2], "%H:%M")
            endtime = datetime.datetime.strptime(train["train_time"][t][1], "%H:%M")
            if (endtime - starttime).seconds / 60 < 0:
                endtime += datetime.timedelta(days=1)
            during = int((endtime - starttime).seconds / 60)
            f_station = transfer(
                train["train_time"][f][3], train["train_time"][f][2], 1
            )
            t_station = transfer(
                train["train_time"][t][3], train["train_time"][t][1], 0
            )
            file.write(
                str(f_station)
                + " "
                + str(t_station)
                + " "
                + str(during)
                + " "
                + train["train_id"]
                + "："
                + train["train_time"][f][0]
                + "至"
                + train["train_time"][t][0]
                + "从"
                + train["train_time"][f][2]
                + "到"
                + train["train_time"][t][1]
                + "\n"
            )

for i in range(total):
    for j in range(station_in(i), station_out(i)):
        for k in range(j + 1, j + 24):
            if k + SPLIT < (i + 1) * SPLIT * 2:
                file.write(
                    str(j)
                    + " "
                    + str(k + SPLIT)
                    + " "
                    + str((k - j) * INTERVAL)
                    + " wait\n"
                )
            else:
                file.write(
                    str(j) + " " + str(k) + " " + str((k - j) * INTERVAL) + " wait\n"
                )

for i in range(total):
    for j in range(station_in(i), station_out(i)):
        file.write(str(j) + " " + str(STATION_END + i) + " 0 end\n")

In [70]:
print(station_out(1))

288
